In [1]:
cd models/research/object_detection

/home/scar3crow/Dropbox/WorkStation-Subrata/python/models/research/object_detection


In [3]:
import numpy as np
import math
import pandas as pd
import cv2
import os
import tqdm
from scipy.io import loadmat

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

from PIL import Image
import pytesseract

import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from keras import backend as K

from utils import *

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam, SGD
from keras.layers import *

from keras.applications import MobileNetV2
from keras.applications import InceptionResNetV2

from keras.models import Model
from keras.models import model_from_json


In [4]:
target_size = [480, 480]
target_w = 480 # target sizes of image in model input
target_h = 480 #target sizes of image in model input

grid_size = [15, 15]
grid_y_axis = 15  # each image is to be segmented to 13 x 13 grid
grid_x_axis = 15  # # each image is to be segmented to 13 x 13 grid

grid_w = target_w / grid_x_axis  # grid cell width
grid_h = target_h / grid_y_axis  # grid cell height

channels = 3
num_anchors = 3
class_num = 5 # vendor, invoice, inv_date, po, buyer
info = 5 + class_num    # pc, x, y, h, w, and class probabilities

categories = ['vendor', 'invoice', 'inv_date', 'po', 'buyer'] # details of classes

In [5]:
# Making a list of image path

inv_directory = '/home/scar3crow/Downloads/8-6-new-scan'  ## 'invoices' is a zip file of jpg images in ...../Downloads 
                                                        
inv_new_image = ['/home/scar3crow/Downloads/8-6-new-scan/{}'.format(i) for i in os.listdir(inv_directory)] # making the list
inv_new_image.sort() # Sorting the list

num_images = len(inv_new_image)

print('Number of images = ', num_images)
inv_new_image[24]

Number of images =  36


'/home/scar3crow/Downloads/8-6-new-scan/50a.jpg'

In [6]:
# Check sizes of exiting images & Create a Dataframe with image id and height(row) and width(column):

rows = []
columns = []
image_sl = []
df_new = pd.DataFrame()

for i in range(len(inv_new_image)):
    image = cv2.imread(inv_new_image[i]) ## Loading image
    height, width, _ = image.shape
    rows.append(height)
    columns.append(width)
    image_sl.append(inv_new_image[i])
    
row_values = pd.Series(rows)
col_values = pd.Series(columns)
image_num = pd.Series(image_sl)


df_new.insert(loc=0, column='image_serial', value=image_num)
df_new.insert(loc=1, column='rows', value=row_values)
df_new.insert(loc=2, column='columns', value=col_values)

df_new.head(3)

,image_serial,rows,columns
0,/home/scar3crow/Downloads/8-6-new-scan/101a.jpg,160,416
1,/home/scar3crow/Downloads/8-6-new-scan/102a.jpg,406,870
2,/home/scar3crow/Downloads/8-6-new-scan/103a.jpg,260,416


In [7]:
# Loading output of VGG Image Annotation tool and create a dataframe

r_new_data = pd.read_csv('/home/scar3crow/Downloads/via_new_data.csv')
num_obj = r_new_data['region_count'][0] # number of objects in each photo
r_new_data.drop(r_new_data.columns[[1, 2, 3, 4]], axis=1, inplace=True) # reduce unnecessary columns
r_new_data.sort_values(by=['#filename'], ascending=True) # Sorting based on image-id
num_images = r_new_data["#filename"].nunique() # Find out number of unique images

print('Number of classes = ', num_obj)
print('Number of unique images = ', num_images)
r_new_data[58:61]

Number of classes =  5
Number of unique images =  36


,#filename,region_shape_attributes,region_attributes
58,63a.jpg,"{""name"":""rect"",""x"":211,""y"":64,""width"":76,""heig...","{""text"":""po""}"
59,63a.jpg,"{""name"":""rect"",""x"":2,""y"":68,""width"":165,""heigh...","{""text"":""buyer""}"
60,101a.jpg,"{""name"":""rect"",""x"":6,""y"":23,""width"":119,""heigh...","{""text"":""vendor""}"


In [8]:
# Making a dataframe for Image_id, x, y, width, height, class, image_width and image_height

x = []
y = []
width = []
height = []
obj_class = []
i_width = []
i_height = []
img_path = []
img_index = []

for i in range(len(r_new_data)):
    
    r_size = r_new_data.values[i, 1][1:(len(r_new_data.values[i, 1])-1)]
    r_size_par = r_size.split(",")
    
    x.append(int("".join(filter(str.isdigit, r_size_par[1]))))
    y.append(int("".join(filter(str.isdigit, r_size_par[2]))))
    width.append(int("".join(filter(str.isdigit, r_size_par[3]))))
    height.append(int("".join(filter(str.isdigit, r_size_par[4]))))
    
    r_attribs = r_new_data.values[i, 2][1:(len(r_new_data.values[i, 2])-1)]
    r_attribs_par = r_attribs.split(':')[1]
    obj_class.append(r_attribs_par[1:(len(r_attribs_par)-1)])
    
    foto_id = r_new_data['#filename'][i]
    i_path = '/home/scar3crow/Downloads/8-6-new-scan/' + foto_id
    foto_index = int(df_new[df_new['image_serial'] == i_path].index[0])
    foto_width = df_new['columns'][foto_index]
    foto_height = df_new['rows'][foto_index]
    i_width.append(foto_width)
    i_height.append(foto_height)
    img_path.append(i_path)
    img_index.append(foto_index)
    
x_values = pd.Series(x)
y_values = pd.Series(y)
width_values = pd.Series(width)
height_values = pd.Series(height)
class_values = pd.Series(obj_class)
i_width_values = pd.Series(i_width)
i_height_values = pd.Series(i_height)
img_path_values = pd.Series(img_path)
img_index_values = pd.Series(img_index)

r_new_data.insert(loc=1, column='img_idx', value=img_index_values)
r_new_data.insert(loc=2, column='i_path', value=img_path_values)
r_new_data.insert(loc=3, column='x', value=x_values)
r_new_data.insert(loc=4, column='y', value=y_values)
r_new_data.insert(loc=5, column='width', value=width_values)
r_new_data.insert(loc=6, column='height', value=height_values)
r_new_data.insert(loc=7, column='obj_class', value=class_values)
r_new_data.insert(loc=8, column='img_wd', value=i_width_values)
r_new_data.insert(loc=9, column='img_ht', value=i_height_values)

r_new_data.drop(r_new_data.columns[[10, 11]], axis=1, inplace=True) # reduce unnecessary columns

r_new_data.rename({'#filename': 'img_id'}, axis=1, inplace=True) # changing column name

r_new_data[3:6]


,img_id,img_idx,i_path,x,y,width,height,obj_class,img_wd,img_ht
3,50a.jpg,24,/home/scar3crow/Downloads/8-6-new-scan/50a.jpg,221,59,103,24,po,416,209
4,50a.jpg,24,/home/scar3crow/Downloads/8-6-new-scan/50a.jpg,5,57,206,56,buyer,416,209
5,51a.jpg,25,/home/scar3crow/Downloads/8-6-new-scan/51a.jpg,5,0,120,56,vendor,416,194


In [9]:
print('Number of unique images = ', r_new_data['img_id'].nunique())  # print total no, of unique images
print('Number of classes in diff. categories = ', r_new_data['obj_class'].value_counts())

Number of unique images =  36
Number of classes in diff. categories =  buyer      38
vendor     36
date       36
invoice    36
po         33
order       1
Name: obj_class, dtype: int64


In [10]:
# We have to correct above :

# To find smallest width & height boxes in 'buyer' which should be 'po'
gb = r_new_data.groupby('obj_class')    
[gb.get_group('buyer') for x in gb.groups]

[       img_id  img_idx                                           i_path    x  \
 4     50a.jpg       24   /home/scar3crow/Downloads/8-6-new-scan/50a.jpg    5   
 9     51a.jpg       25   /home/scar3crow/Downloads/8-6-new-scan/51a.jpg    4   
 14    52a.jpg       26   /home/scar3crow/Downloads/8-6-new-scan/52a.jpg    1   
 19    53a.jpg       27   /home/scar3crow/Downloads/8-6-new-scan/53a.jpg    0   
 24    54a.jpg       28   /home/scar3crow/Downloads/8-6-new-scan/54a.jpg   31   
 29    55a.jpg       29   /home/scar3crow/Downloads/8-6-new-scan/55a.jpg    1   
 34    56a.jpg       30   /home/scar3crow/Downloads/8-6-new-scan/56a.jpg    1   
 39    59a.jpg       31   /home/scar3crow/Downloads/8-6-new-scan/59a.jpg    3   
 44    60a.jpg       32   /home/scar3crow/Downloads/8-6-new-scan/60a.jpg    0   
 49    61a.jpg       33   /home/scar3crow/Downloads/8-6-new-scan/61a.jpg    1   
 54    62a.jpg       34   /home/scar3crow/Downloads/8-6-new-scan/62a.jpg    4   
 59    63a.jpg       35   /h

In [11]:
# Correcting above wrong spelling & converting buyer to po of object classes and rechecking

id_1 = r_new_data.index[r_new_data['obj_class'] == 'order'] # Finding the index
id_2 = r_new_data.index[r_new_data['obj_class'] == 'date'] # to change 'date' to 'inv_date' to be consistent with old data

r_new_data.at[id_1, 'obj_class'] = 'po' # writing the correct spelling 
r_new_data.at[88, 'obj_class'] = 'po' # # 'buyer' to 'po'
r_new_data.at[163, 'obj_class'] = 'po' # # 'buyer' to 'po'
r_new_data.at[id_2, 'obj_class'] = 'inv_date' # # 'date' to 'inv_date'

print('Number of unique images = ', r_new_data['img_id'].nunique())  # print total no, of unique images
print('Number of unique classes = ', r_new_data['obj_class'].nunique())
print('Number of classes in diff. categories = ', r_new_data['obj_class'].value_counts())

# r_new_data.drop(r_new_data.columns[[0]], axis=1, inplace=True) # reduce unnecessary columns

Number of unique images =  36
Number of unique classes =  5
Number of classes in diff. categories =  vendor      36
inv_date    36
po          36
invoice     36
buyer       36
Name: obj_class, dtype: int64


In [12]:
## For each image, we have to find : (a) line_index = integer, (b) img_path = string, (c) boxes = shape [N, 4], 
## N is the ground truth count, elements in the second dimension are [x_min, y_min, x_max, y_max] (d) labels = shape
## [N]. class index. (e) img_width = int.  =f) img_height = int

def single_image_info(lines):
    
    ## lines will be a dataframe like, for i in range(num_images), lines = r_new_data[i*5:(i+1)*5]
    
    line_idx = lines.iat[0, 1]
    pic_path = lines.iat[0, 2]
    img_width = lines.iat[0, 8]
    img_height = lines.iat[0, 9]
    
    boxes = []
    labels = []
    for i in range(len(lines)):
        label, x_min, y_min, x_max, y_max = int(i), float(lines.iat[i,3]), float(lines.iat[i,4]), float(lines.iat[i,3]+lines.iat[i,5]), float(lines.iat[i,4]+lines.iat[i,6])
        boxes.append([x_min, y_min, x_max, y_max])
        labels.append(label)
        
    boxes = np.asarray(boxes, np.float32)
    labels = np.asarray(labels, np.int64)
    
    return line_idx, pic_path, boxes, labels, img_width, img_height  ## boxes are in format xmin, ymin, xmax, ymax


In [13]:
## Creating the complete data set :

all_image_line = []
for i in range(num_images):
    image_line = []
    limit_lower = i*5
    limit_upper = limit_lower+5
    lines = r_new_data[limit_lower:limit_upper]
    line_idx, pic_path, boxes, labels, img_width, img_height = single_image_info(lines)
    image_line.append(line_idx)
    image_line.append(pic_path)
    image_line.append(boxes)
    image_line.append(labels)
    image_line.append(img_width)
    image_line.append(img_height)
    all_image_line.append(image_line)
    
print(len(all_image_line))
print(all_image_line[1])   ##  boxes are in format xmin, ymin, xmax, ymax

36
[25, '/home/scar3crow/Downloads/8-6-new-scan/51a.jpg', array([[  5.,   0., 125.,  56.],
       [239.,   1., 279.,  20.],
       [328.,   1., 382.,  21.],
       [238.,  51., 302.,  74.],
       [  4.,  53., 156., 117.]], dtype=float32), array([0, 1, 2, 3, 4]), 416, 194]


In [14]:
## Train and Test split

data_train, data_val = train_test_split(all_image_line, train_size = 0.8 , shuffle = True)

num_all_bbox = len(all_image_line) * len(all_image_line[0][2])
num_bb_train = len(data_train) * len(data_train[0][2])
num_bb_val = len(data_val) * len(data_val[0][2])
print(num_all_bbox, num_bb_train, num_bb_val)

180 140 40


/home/scar3crow/Dropbox/WorkStation-Subrata/python/venv1/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [15]:
## calculating anchors from true boundary boxes :

def iou_kmeans(box, clusters):
    """
    Calculates the Intersection over Union (IoU) between a box and k clusters.
    :param box: tuple or array, shifted to the origin (i. e. width and height)
    :param clusters: numpy array of shape (k, 2) where k is the number of clusters
    :return: numpy array of shape (k, 0) where k is the number of clusters
    """
    x = np.minimum(clusters[:, 0], box[0])
    y = np.minimum(clusters[:, 1], box[1])
    if np.count_nonzero(x == 0) > 0 or np.count_nonzero(y == 0) > 0:
        raise ValueError("Box has no area")

    intersection = x * y
    box_area = box[0] * box[1]
    cluster_area = clusters[:, 0] * clusters[:, 1]

    iou = intersection / (box_area + cluster_area - intersection)

    return iou

def kmeans(boxes, k, dist=np.median):
    """
    Calculates k-means clustering with the Intersection over Union (IoU) metric.
    :param boxes: numpy array of shape (r, 2), where r is the number of rows
    :param k: number of clusters
    :param dist: distance function
    :return: numpy array of shape (k, 2)
    """
    rows = boxes.shape[0]

    distances = np.empty((rows, k))
    last_clusters = np.zeros((rows,))

    np.random.seed()

    # the Forgy method will fail if the whole array contains the same rows
    clusters = boxes[np.random.choice(rows, k, replace=False)]


    while True:
        for row in range(rows):
            distances[row] = 1 - iou_kmeans(boxes[row], clusters)

        nearest_clusters = np.argmin(distances, axis=1)

        if (last_clusters == nearest_clusters).all():
            break

        for cluster in range(k):
            clusters[cluster] = dist(boxes[nearest_clusters == cluster], axis=0)

        last_clusters = nearest_clusters

    return clusters

In [172]:
## Finding out anchors :
## Firstly, converting true boundary box width, height to width & height with respect to target image :
## finaly find anchors. Anchors here are in absolute size w.r.t. target image but not as % of target image or 
## as multiple of unit grids.

# num_all_bb = len(r_new_data) # if no. of bboxes varies for images, this formula should be used 

anchors_wrt_target = np.zeros((3,2))

num_all_bb = len(all_image_line) * len(all_image_line[0][2])  ## from all image line data

b_box_wrt_target = np.zeros((num_all_bb,2))

for i in range(num_all_bb):
    
    image_w = r_new_data['img_wd'][i]
    image_h = r_new_data['img_ht'][i]

    x_ratio = target_w / image_w 
    y_ratio = target_h / image_h
    
    anchor_w = r_new_data['width'][i] * x_ratio
    anchor_h = r_new_data['height'][i] * y_ratio
    b_box_wrt_target[i, 0] = anchor_w
    b_box_wrt_target[i, 1] = anchor_h
    
anchors_wrt_target = kmeans(b_box_wrt_target, num_anchors)

print(anchors_wrt_target.shape)
print(anchors_wrt_target)     ## anchors wrt target image in abs. value and in format width, height


(3, 2)
[[189.80769231 160.36697248]
 [ 68.07692308  65.30612245]
 [129.23076923  37.92592593]]


In [22]:
## Pre-processing the original data to get y_true :

def process_box(ori_boxes, ori_img_width, ori_img_height, labels, target_size, class_num, anchors):
    '''
    Generate the y_true label, i.e. the ground truth feature_map.
    params:
        boxes: [N, 5] shape, float32 dtype. `x_min, y_min, x_max, y_mix, mixup_weight`.
        labels: [N] shape, int64 dtype.
        class_num: int64 num.
        anchors: [3,2] shape, float32 dtype.
    '''
    
    img_width = ori_img_width
    img_height = ori_img_height
    boxes = ori_boxes           ## boxes in format xmin, ymin, xmax, ymax
    
    x_ratio = target_size[1] / img_width
    y_ratio = target_size[0] / img_height
    
    boxes_wrt_target = np.zeros((5,4))
    box_centers_target = np.zeros((5,2))

    boxes_wrt_target[:,0] = boxes[:,0] * x_ratio  # xmin absolute value wrt target image
    boxes_wrt_target[:,1] = boxes[:,1] * y_ratio  # ymin absolute value wrt target image
    boxes_wrt_target[:,2] = boxes[:,2] * x_ratio  # xmax absolute value wrt target image
    boxes_wrt_target[:,3] = boxes[:,3] * y_ratio  # ymax absolute value wrt target image
    
    # In above, boxes_wrt_target shape is (5, 4), now this will be taken to (5. 5) by adding 1 at end
#    boxes_wrt_target = np.concatenate((boxes_wrt_target, np.full(shape=(boxes_wrt_target.shape[0], 1), fill_value=1., dtype=np.float32)), axis=-1)
    box_centers_target = (boxes_wrt_target[:, 0:2] + boxes_wrt_target[:, 2:4]) / 2  ## centers wrt target, abs values
    
    box_sizes = boxes[:, 2:4] - boxes[:, 0:2]  #xmax-xmin = width and ymax-ymin = height wrt original image
    box_sizes[:,0] = box_sizes[:,0] * x_ratio  # width w.r.t target image in absolute value
    box_sizes[:,1] = box_sizes[:,1] * y_ratio  # width w.r.t target image in absolute value
    
#    y_true_13 = np.zeros((target_size[1] // 32, target_size[0] // 32, 3, 6 + class_num), np.float32)
    y_true_13 = np.zeros((target_size[1] // 32, target_size[0] // 32, 3, 5 + class_num), np.float32)

#    y_true = [y_true_13]
    
    box_sizes = np.expand_dims(box_sizes, 1)
    mins = np.maximum(- box_sizes / 2, - anchors / 2)
    maxs = np.minimum(box_sizes / 2, anchors / 2)
    whs = maxs - mins

    iou = (whs[:, :, 0] * whs[:, :, 1]) / (
                box_sizes[:, :, 0] * box_sizes[:, :, 1] + anchors[:, 0] * anchors[:, 1] - whs[:, :, 0] * whs[:, :,
                                                                                                         1] + 1e-10)
    best_match_idx = np.argmax(iou, axis=1)

    anchor_mask = np.zeros((target_size[1] // 32, target_size[0] // 32, 3))

    cell_size = 32  ## = targetsize / no. of grid cells
    
    for i, idx in enumerate(best_match_idx):

        x = int(np.floor(box_centers_target[i, 0] / cell_size))
        y = int(np.floor(box_centers_target[i, 1] / cell_size))
        k = int(idx)
        c = int(labels[i])

        print(x, y, k, c)

# Very Imp : Now preparing y_true: all values x_center, y_cemter, width & height are being taken to % of target image
        
        y_true_13[y, x, k, :2] = box_centers_target[i] / target_size[0] #since target_size[0] = target_size[1]
        y_true_13[y, x, k, 2:4] = box_sizes[i] / target_size[0]
        y_true_13[y, x, k, 4] = 1.
        y_true_13[y, x, k, 5 + c] = 1.
#        y_true[0][y, x, k, -1] = boxes_wrt_target[i, -1]
        anchor_mask[y, x, k] = 1

    return y_true_13, anchor_mask  ## all data are w.r.to target image in % of target image and NOT w,r,t, grid cells

In [23]:
## Single image-wise image/boundary box preprocessing:

def parse_data(line, class_num, target_size, anchors):   ## (mode, letterbox_resize):
    '''
    param:
        line: a line from the training/test txt file
        class_num: totol class nums.
        target_size: the size of image to be resized to. [width, height] format.
        anchors: anchors.
        mode: 'train' or 'val'. When set to 'train', data_augmentation will be applied.
        letterbox_resize: whether to use the letterbox resize, i.e., keep the original aspect ratio in the resized image.
    '''
    
    img_idx, pic_path, boxes, labels,img_width, img_height = line  # boxes in format xmin, ymin, xmax, ymax
    img = cv2.imread(pic_path)
    img_resized = cv2.resize(img,(target_size[0], target_size[1]))
    
    # expand the 2nd dimension, mix up weight default to 1.
    boxes = np.concatenate((boxes, np.full(shape=(boxes.shape[0], 1), fill_value=1., dtype=np.float32)), axis=-1)

    img_resized = cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB).astype(np.float32)

    # the input of yolo_v3 should be in range 0~1, lets change to -0.5 to +0.5
    img_resized = (img_resized - 127.5)/ 255.

    y_true_13, anchor_mask = process_box(boxes, img_width, img_height, labels, target_size, class_num, anchors)

    return img_idx, img_resized, y_true_13, anchor_mask


In [24]:
## Making the data ready for entering into network :

anchors = anchors_wrt_target
image_index = []
image_resized = []
image_y_true = []
image_anchor_mask = []

for i in range(len(data_train)):

    line = data_train[i]
    
    img_idx, img_resized, y_true, anchor_mask = parse_data(line, class_num, target_size, anchors)
    
    
    image_index.append(img_idx)
    image_resized.append(img_resized)
    image_y_true.append(y_true)
    image_anchor_mask.append(anchor_mask)
    
train_image_index = image_index
X_train = np.array(image_resized)
Y_train = np.array(image_y_true)
train_anchor_mask = np.array(image_anchor_mask)

image_index = []
image_resized = []
image_y_true = []
image_anchor_mask = []

for i in range(len(data_val)):
    line = data_val[i]
    
    img_idx, img_resized, y_true, anchor_mask = parse_data(line, class_num, target_size, anchors)
    image_index.append(img_idx)
    image_resized.append(img_resized)
    image_y_true.append(y_true)
    image_anchor_mask.append(anchor_mask)
val_image_index = image_index
X_val = np.array(image_resized)
Y_val = np.array(image_y_true)
val_anchor_mask = np.array(image_anchor_mask)

image_index = []
image_resized = []
image_y_true = []


2 2 2 0
9 2 0 1
12 2 0 2
9 8 0 3
2 9 2 4
5 2 1 0
2 5 0 1
2 4 0 2
2 7 0 3
10 12 2 4
3 2 2 0
9 1 0 1
12 1 0 2
9 5 0 3
2 6 2 4
7 5 1 0
10 10 0 1
10 11 0 2
11 13 0 3
4 12 1 4
2 2 2 0
9 1 0 1
12 1 0 2
9 6 0 3
2 9 2 4
2 3 2 0
9 1 0 1
12 1 0 2
9 8 0 3
3 10 2 4
3 2 2 0
9 0 0 1
12 0 0 2
10 5 0 3
3 6 2 4
2 3 2 0
9 5 0 1
9 7 0 2
9 12 0 3
3 9 1 4
7 3 1 0
10 9 0 1
11 11 0 2
11 12 0 3
4 11 2 4
2 2 2 0
9 1 0 1
12 1 0 2
9 5 0 3
3 7 2 4
11 3 2 0
9 9 0 1
13 9 0 2
10 12 0 3
4 11 2 4
11 4 2 0
9 9 0 1
13 9 0 2
9 12 0 3
3 12 2 4
2 5 2 0
8 3 0 1
12 3 0 2
9 8 0 3
3 11 2 4
2 2 2 0
9 1 0 1
12 0 0 2
9 5 0 3
3 6 2 4
7 2 1 0
11 9 0 1
11 10 0 2
11 12 0 3
4 11 2 4
2 2 2 0
8 1 0 1
12 1 0 2
8 6 0 3
3 7 2 4
2 3 2 0
8 1 0 1
12 1 0 2
9 7 0 3
2 10 2 4
6 2 1 0
2 11 0 1
2 12 0 2
7 11 0 3
10 8 2 4
2 3 2 0
9 1 0 1
12 1 0 2
10 7 0 3
3 9 2 4
2 2 2 0
9 1 0 1
12 2 0 2
10 7 0 3
2 7 2 4
2 3 2 0
9 1 0 1
12 1 0 2
10 7 0 3
3 11 2 4
4 2 2 0
9 1 0 1
12 0 0 2
10 4 0 3
3 7 2 4
2 2 2 0
9 0 0 1
12 0 0 2
9 3 0 3
3 6 2 4
3 2 1 0
9 1 0 1
12 1 

In [25]:
print(len(X_train), len(Y_train), len(train_image_index), len(X_val), len(Y_val), len(val_image_index))
print(train_image_index)
print(X_train.shape, Y_train.shape)
print(train_image_index[0], X_train[0].shape, Y_train[0].shape)
print(train_anchor_mask.shape, 'and', train_anchor_mask[0].shape)

28 28 28 8 8 8
[5, 7, 26, 9, 4, 34, 27, 12, 28, 14, 10, 3, 13, 16, 21, 35, 33, 1, 29, 6, 22, 17, 31, 11, 23, 20, 19, 30]
(28, 480, 480, 3) (28, 15, 15, 3, 10)
5 (480, 480, 3) (15, 15, 3, 10)
(28, 15, 15, 3) and (15, 15, 3)


In [26]:
# https://github.com/ethanyanjiali/deep-vision/blob/master/YOLO/tensorflow/utils.py

def xywh_to_x1y1x2y2(box):
    xy = box[..., 0:2]
    wh = box[..., 2:4]

    x1y1 = xy - wh / 2
    x2y2 = xy + wh / 2

    y_box = K.concatenate([x1y1, x2y2], axis=-1)
    return y_box

In [27]:
# https://github.com/ethanyanjiali/deep-vision/blob/master/YOLO/tensorflow/utils.py

def broadcast_iou(box_a, box_b):
    """
    calculate iou between box_a and multiple box_b in a broadcast way
    inputs: box_a: a tensor full of boxes, eg. (B, N, 4), box is in x1y1x2y2
            box_b: another tensor full of boxes, eg. (B, M, 4)
    """

    # (B, N, 1, 4)
    box_a = tf.expand_dims(box_a, -2)
    # (B, 1, M, 4)
    box_b = tf.expand_dims(box_b, -3)
    # (B, N, M, 4)
    new_shape = tf.broadcast_dynamic_shape(tf.shape(box_a), tf.shape(box_b))

    # (B, N, M, 4)
    # (B, N, M, 4)
    box_a = tf.broadcast_to(box_a, new_shape)
    box_b = tf.broadcast_to(box_b, new_shape)

    # (B, N, M, 1)
    al, at, ar, ab = tf.split(box_a, 4, -1)
    bl, bt, br, bb = tf.split(box_b, 4, -1)

    # (B, N, M, 1)
    left = tf.math.maximum(al, bl)
    right = tf.math.minimum(ar, br)
    top = tf.math.maximum(at, bt)
    bot = tf.math.minimum(ab, bb)

    # (B, N, M, 1)
    iw = tf.clip_by_value(right - left, 0, 1)
    ih = tf.clip_by_value(bot - top, 0, 1)
    i = iw * ih

    # (B, N, M, 1)
    area_a = (ar - al) * (ab - at)
    area_b = (br - bl) * (bb - bt)
    union = area_a + area_b - i

    # (B, N, M)
    iou = tf.squeeze(i / (union + 1e-7), axis=-1)

    return iou


In [28]:
## https://github.com/ethanyanjiali/deep-vision/blob/master/YOLO/tensorflow/yolov3.py#L213

def calc_ignore_mask(ignore_thresh, true_box, pred_box):
    
        # YOLOv3:
        # "If the bounding box prior is not the best but does overlap a ground
        # truth object by more than some threshold we ignore the prediction,
        # following [17]. We use the threshold of .5."
        # calculate the iou for each pair of pred bbox and true bbox, then find the best among them

        # (None, 13, 13, 3, 4)
        
        true_box_reorganised = xywh_to_x1y1x2y2(true_box)  # reorganised to x1, y1, x2, y2
        pred_box_reorganised = xywh_to_x1y1x2y2(pred_box)
        
        true_box_shape = tf.shape(true_box_reorganised)  
        # (None, 13, 13, 3, 4)
        pred_box_shape = tf.shape(pred_box_reorganised)  
        # (None, 507, 4)
        true_box_reorganised = tf.reshape(true_box_reorganised, [true_box_shape[0], -1, 4])
        # sort true_box to have non-zero boxes rank first
        true_box_reorganised = tf.sort(true_box_reorganised, axis=1, direction="DESCENDING")
        # (None, 100, 4)
        # only use maximum 100 boxes per groundtruth to calcualte IOU, otherwise
        # GPU emory comsumption would explode for a matrix like (16, 52*52*3, 52*52*3, 4)
        true_box_reorganised = true_box_reorganised[:, 0:100, :]
        # (None, 507, 4)
        pred_box_reorganised = tf.reshape(pred_box_reorganised, [pred_box_shape[0], -1, 4])

        # https://github.com/dmlc/gluon-cv/blob/06bb7ec2044cdf3f433721be9362ab84b02c5a90/gluoncv/model_zoo/yolo/yolo_target.py#L198
        # (None, 507, 507)
        iou = broadcast_iou(pred_box_reorganised, true_box_reorganised)
        # (None, 507)
        best_iou = tf.reduce_max(iou, axis=-1)
        # (None, 13, 13, 3)
        best_iou = tf.reshape(best_iou, [pred_box_shape[0], pred_box_shape[1], pred_box_shape[2], pred_box_shape[3]])
        # ignore_mask = 1 => don't ignore
        # ignore_mask = 0 => should ignore
        ignore_mask = tf.cast(best_iou < ignore_thresh, tf.float32)
        # (None, 13, 13, 3, 1)
        ignore_mask = tf.expand_dims(ignore_mask, axis=-1)
        
        return ignore_mask

In [89]:
def binary_cross_entropy(logits, labels):
    epsilon = 1e-7
    logits = tf.clip_by_value(logits, epsilon, 1 - epsilon)
    return -(labels * tf.math.log(logits) +
             (1 - labels) * tf.math.log(1 - logits))


In [29]:
# def yolo_my_loss(y_true, y_pred, anchors_wrt_target):
def yolo_my_loss(y_true, y_pred):
    
    '''Return yolo_loss
    Parameters
    ----------
    yolo_outputs: list of tensor, the output of yolo_body or tiny_yolo_body
    y_true: list of array, the output of preprocess_true_boxes
    anchors: array, shape=(N, 2), wh ## w.r.t. target size
    num_classes: integer
    ignore_thresh: float, the iou threshold whether to ignore object confidence loss
    Returns
    -------
    loss: tensor, shape=(1,)
    '''
    grid_size = [15., 15.]
    ratio = 480./15.      ## ratio = 32
    ignore_thresh = 0.5
    Lambda_Coord = 5.0
    Lambda_no_obj = 0.5
    
    grid_x = np.arange(grid_size[1])
    grid_y = np.arange(grid_size[0])
    
    a = np.array(np.meshgrid(grid_x, grid_y))
    b = np.array(np.meshgrid(grid_x, grid_y))
    c = np.array(np.meshgrid(grid_x, grid_y))
    d = np.concatenate((a,b,c), axis = 0)
    e = d.transpose(2, 1, 0)
    grid_final = np.reshape(e,[15,15,3,2])
    
#    grid_x = np.arange(grid_size[1])
#    grid_y = np.arange(grid_size[0])
#    grid_x, grid_y = np.meshgrid(grid_x, grid_y)
#    grid_1st = np.dstack((grid_x, grid_y))
#    grid_2nd = np.dstack((grid_1st, grid_x, grid_y))
#    grid_3rd = np.dstack((grid_2nd, grid_x, grid_y)) 
#    grid_final = np.reshape(grid_3rd, [1,15,15,3,2])
    
    loss = tf.zeros(1, dtype='float32')
    
    
#    m = y_true.shape[0]
#    m = np.expand_dims(m, axis=-1)
#    mf = K.cast(m, dtype='float32')

    
    obj_mask_y = y_true[..., 4:5]
#    obj_mask = K.cast(obj_mask_y, dtype='float32')
    
#    true_box_wh = y_true[..., 2:4]
#    weight = 2 - true_box_wh[..., 0] * true_box_wh[..., 1]
#    weight = np.expand_dims(weight, axis=-1)
    
## ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++    
#    pred_box_xy = K.sigmoid(y_pred[..., :2]) + grid_final  # this gives x & y in no. of cells. x & y w.r.t. target
                                                            # image = (x & y in no. of cells) / no. of cells
    pred_box_xy = my_sigmoid(y_pred[..., :2]) + grid_final  # this gives x & y in no. of cells. x & y w.r.t. target
                                                            # image = (x & y in no. of cells) / no. of cells
        
        
        
#    Lambda_Coord = K.cast(Lambda_Coord, dtype = 'float32')
    pred_box_xy_wrt_target_image = (pred_box_xy * ratio) / 480.
    true_box_xy_wrt_target_image = y_true[..., :2]
    
#    pred_box_xy_wrt_ti = K.cast(pred_box_xy_wrt_target_image, dtype = 'float32')
#    true_box_xy_wrt_ti = K.cast(true_box_xy_wrt_target_image, dtype = 'float32')
    
#    xy_arr = K.cast((true_box_xy_wrt_ti - pred_box_xy_wrt_ti), dtype='float32')

    xy_arr = np.power((true_box_xy_wrt_ti - pred_box_xy_wrt_ti), 2)
    
    xy_loss = Lambda_Coord * np.sum(xy_arr * obj_mask_y)
    
#    xy_loss = K.cast(Lambda_Coord * tf.reduce_sum(tf.square(xy_arr) * obj_mask), dtype = 'float32')
    
#    xy_loss = (Lambda_Coord *np.sum(mean_squared_error(true_box_xy_wrt_ti, pred_box_xy_wrt_ti) * obj_mask*weight)) / m
    
#    xy_loss = (Lambda_Coord *np.sum(mean_squared_error(true_box_xy_wrt_ti, pred_box_xy_wrt_ti) * obj_mask*weight)) / m
    
#    xy_loss = (Lambda_Coord*tf.reduce_sum(tf.square(true_box_xy_wrt_ti - pred_box_xy_wrt_ti) *obj_mask))/m
#    wh_loss = tf.reduce_sum(tf.square(true_tw_th - pred_tw_th) * object_mask * box_loss_scale * mix_w) / N
    
## ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++    
#    pred_box_wdht = K.exp(y_pred[..., 2:4]) * (np.reshape(anchors_wrt_target, [1,1,1,3,2])/480.)
    
    pred_box_wdht = np.exp(y_pred[..., 2:4]) * (anchors_wrt_target / 480.)
    
#    pred_box_wh = K.cast(pred_box_wdht, dtype = 'float32')
    true_box_wdht = y_true[..., 2:4]
#    true_box_wh = K.cast(true_box_wdht, dtype = 'float32')
    
#    wh_arr = K.cast((K.sqrt(true_box_wh) - K.sqrt(pred_box_wh)), dtype='float32')
    
    wh_arr = np.power((true_box_wdht - pred_box_wdht), 2)
    
    wh_loss = Lambda_Coord * np.sum(wh_arr * obj_mask_y)
    
## ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ 

    pred_obj_mask = K.cast(K.sigmoid(y_pred[..., 4:5]), dtype = 'float32')  # shape = 28, 15, 15, 3, 1
       
    true_box_wrt_ti = K.concatenate([true_box_xy_wrt_ti, true_box_wh], axis = -1)  ## in x,y,w,h format
    pred_box_wrt_ti = K.concatenate([pred_box_xy_wrt_ti, pred_box_wh], axis = -1)  ## in x,y,w,h format
    
    
    
    
    true_box_wrt_ti = xywh_to_x1y1x2y2(true_box_wrt_ti)  ## converted to x1,y1,x2,y2 format
    pred_box_wrt_ti = xywh_to_x1y1x2y2(pred_box_wrt_ti)  ## converted to x1,y1,x2,y2 format
    
    ignore_mask = calc_ignore_mask(ignore_thresh, true_box_wrt_ti, pred_box_wrt_ti)
    
    ignore_mask = K.cast(ignore_mask, dtype = 'float32')                               

    obj_loss = K.cast(K.sum(K.binary_crossentropy(obj_mask, pred_obj_mask) * obj_mask), dtype = 'float32')
    
    no_obj_mask = 1. - obj_mask_y
    no_obj_mask = K.cast(no_obj_mask, dtype  = 'float32')
    
    noobj_loss = K.cast(Lambda_no_obj * K.sum(binary_cross_entropy(obj_mask, pred_obj_mask) * no_obj_mask * ignore_mask), dtype='float32')
    
## ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

    true_classes = K.cast(y_true[..., 5:10], dtype = 'float32')
    
    pred_classes = K.cast(y_pred[..., 5:10], dtype = 'float32')
    
    class_loss = K.cast(K.sum(binary_cross_entropy(true_classes, pred_classes) * obj_mask), dtype = 'float32')

    loss = xy_loss + wh_loss + obj_loss + noobj_loss + class_loss
    
    return loss

In [27]:
def yolo_model(input_shape):
    
    
    inp = Input(input_shape)
   
    model = InceptionResNetV2( input_tensor= inp , include_top=False, weights='imagenet')
    last_layer = model.output
    
    new_last_layer = UpSampling2D(2)(last_layer)
    
    conv = Conv2D(512,(3,3) , padding='valid')(new_last_layer)
    conv = Dropout(0.4)(conv)
    bn = BatchNormalization()(conv)
    lr = LeakyReLU(alpha=0.1)(bn)
    
    conv = Conv2D(256,(3,3) , padding='valid')(lr)
    conv = Dropout(0.4)(conv)
    bn = BatchNormalization()(conv)
    lr = LeakyReLU(alpha=0.1)(bn)
    
    another_up_layer = UpSampling2D(2)(lr)
    
    conv = Conv2D(256,(3,3) , padding='valid')(another_up_layer)
    conv = Dropout(0.4)(conv)
    bn = BatchNormalization()(conv)
    lr = LeakyReLU(alpha=0.1)(bn)
    
    conv = Conv2D(256,(3,3) , padding='valid')(lr)
    conv = Dropout(0.4)(conv)
    bn = BatchNormalization()(conv)
    lr = LeakyReLU(alpha=0.1)(bn)
    
    conv = Conv2D(128,(3,3) , strides = (2, 2), padding='valid')(lr)
    conv = Dropout(0.4)(conv)
    bn = BatchNormalization()(conv)
    lr = LeakyReLU(alpha=0.1)(bn)
    
    conv = Conv2D(64,(3,3) , padding='valid')(lr)
    conv = Dropout(0.4)(conv)
    bn = BatchNormalization()(conv)
    lr = LeakyReLU(alpha=0.1)(bn)
    
    conv = Conv2D(64,(3,3) , padding='valid')(lr)
    conv = Dropout(0.4)(conv)
    bn = BatchNormalization()(conv)
    lr = LeakyReLU(alpha=0.1)(bn)
    
    conv = Conv2D(30,(1,1) , padding='same')(lr)
#    conv = Dropout(0.4)(conv)
#    bn = BatchNormalization()(conv)
#    lr = LeakyRelu(alpha=0.1)(bn)

    final = Reshape((grid_y_axis,grid_x_axis,num_anchors,info))(conv)

    model = Model(inp,final)
    features = model.output

    return model, features

input_size = (target_w, target_h, 3)
my_yolo_invoice_model, features_1 = yolo_model(input_size)


my_model_1, features_2 = yolo_model(input_size)
my_model_2, features_3 = yolo_model(input_size)

print(my_model_1.summary())




Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 480, 480, 3)  0                                            
__________________________________________________________________________________________________
conv2d_212 (Conv2D)             (None, 239, 239, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_211 (BatchN (None, 239, 239, 32) 96          conv2d_212[0][0]                 
__________________________________________________________________________________________________
activation_204 (Activation)     (None, 239, 239, 32) 0           batch_normalization_211[0][0]    
__________________________________________________________________________________________

In [30]:
opt = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
my_model_1.compile(optimizer= opt,loss=yolo_my_loss,metrics=['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [31]:
my_model_1.fit(X_train ,Y_train ,epochs= 5,batch_size = 4, validation_data=(X_val,Y_val))


Train on 28 samples, validate on 8 samples
Epoch 1/5
28/28 [==============================] - 269s 10s/step - loss: 10310.7928 - accuracy: 0.0768 - val_loss: 9228.9614 - val_accuracy: 0.1407
Epoch 2/5
28/28 [==============================] - 140s 5s/step - loss: 9733.9142 - accuracy: 0.0755 - val_loss: 9493.9355 - val_accuracy: 0.1487
Epoch 3/5
28/28 [==============================] - 135s 5s/step - loss: 9667.6391 - accuracy: 0.0747 - val_loss: 9857.2954 - val_accuracy: 0.1241
Epoch 4/5
28/28 [==============================] - 133s 5s/step - loss: 9296.8830 - accuracy: 0.0716 - val_loss: 9961.8042 - val_accuracy: 0.1152
Epoch 5/5
28/28 [==============================] - 134s 5s/step - loss: 9237.8970 - accuracy: 0.0738 - val_loss: 9854.7388 - val_accuracy: 0.1257


In [35]:
opt = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
my_model_2.compile(optimizer= opt,loss=yolo_my_loss,metrics=['accuracy'])

In [36]:
my_model_2.fit(X_train ,Y_train ,epochs= 5,batch_size = 4, validation_data=(X_val,Y_val))

Train on 28 samples, validate on 8 samples
Epoch 1/5
28/28 [==============================] - 732s 26s/step - loss: 10666.9745 - accuracy: 0.1079 - val_loss: 9359.6025 - val_accuracy: 0.0365
Epoch 2/5
28/28 [==============================] - 133s 5s/step - loss: 10244.0806 - accuracy: 0.1169 - val_loss: 9797.1753 - val_accuracy: 0.0254
Epoch 3/5
28/28 [==============================] - 136s 5s/step - loss: 9903.6975 - accuracy: 0.1272 - val_loss: 10586.7803 - val_accuracy: 0.0130
Epoch 4/5
28/28 [==============================] - 134s 5s/step - loss: 9655.2091 - accuracy: 0.1386 - val_loss: 11212.1104 - val_accuracy: 0.0150
Epoch 5/5
28/28 [==============================] - 133s 5s/step - loss: 9311.2352 - accuracy: 0.1396 - val_loss: 11281.1006 - val_accuracy: 0.0344


In [37]:
my_model_2.fit(X_train ,Y_train ,epochs= 20,batch_size = 4, validation_data=(X_val,Y_val))

Train on 28 samples, validate on 8 samples
Epoch 1/20
28/28 [==============================] - 126s 5s/step - loss: 9160.9930 - accuracy: 0.1404 - val_loss: 11221.5405 - val_accuracy: 0.0583
Epoch 2/20
28/28 [==============================] - 134s 5s/step - loss: 8766.9420 - accuracy: 0.1396 - val_loss: 11242.6411 - val_accuracy: 0.0669
Epoch 3/20
28/28 [==============================] - 132s 5s/step - loss: 8429.4667 - accuracy: 0.1294 - val_loss: 11125.7871 - val_accuracy: 0.0806
Epoch 4/20
28/28 [==============================] - 135s 5s/step - loss: 8215.1627 - accuracy: 0.1303 - val_loss: 10941.2134 - val_accuracy: 0.0856
Epoch 5/20
28/28 [==============================] - 136s 5s/step - loss: 8834.7207 - accuracy: 0.1207 - val_loss: 10758.3223 - val_accuracy: 0.0722
Epoch 6/20
28/28 [==============================] - 138s 5s/step - loss: 7700.3325 - accuracy: 0.1150 - val_loss: 10238.6450 - val_accuracy: 0.0376
Epoch 7/20
28/28 [==============================] - 134s 5s/step - lo

In [42]:

def my_sigmoid(x):
    result = 1/(1 + np.exp(-x))
    return result

In [38]:
grid_size = [15., 15.]
ratio = 480./15.      ## ratio = 32
ignore_thresh = 0.5
Lambda_Coord = 5.0
Lambda_no_obj = 0.5
    
grid_x = np.arange(grid_size[1])
grid_y = np.arange(grid_size[0])
    
a = np.array(np.meshgrid(grid_x, grid_y))
b = np.array(np.meshgrid(grid_x, grid_y))
c = np.array(np.meshgrid(grid_x, grid_y))
d = np.concatenate((a,b,c), axis = 0)
e = d.transpose(2, 1, 0)
grid_final = np.reshape(e,[15,15,3,2])

print(grid_final[1,0])

[[1. 0.]
 [1. 0.]
 [1. 0.]]


In [30]:
id_train0 = r_new_data.index[r_new_data['img_idx'] == 5] # Finding the index
#idd = all_image_line.index[all_image_line[0] == 26]

print(id_train0)

Int64Index([85, 86, 87, 88, 89], dtype='int64')


In [36]:
all_image_line[17] # here box is in x1, y1, x2, y2 format as per original image

[5,
 '/home/scar3crow/Downloads/8-6-new-scan/106a.jpg',
 array([[  2.,   1., 130.,  52.],
        [231.,   9., 271.,  30.],
        [323.,   9., 373.,  31.],
        [230.,  63., 319.,  85.],
        [  1.,  53., 155., 125.]], dtype=float32),
 array([0, 1, 2, 3, 4]),
 416,
 134]

In [33]:
r_new_data[85:90]

,img_id,img_idx,i_path,x,y,width,height,obj_class,img_wd,img_ht
85,106a.jpg,5,/home/scar3crow/Downloads/8-6-new-scan/106a.jpg,2,1,128,51,vendor,416,134
86,106a.jpg,5,/home/scar3crow/Downloads/8-6-new-scan/106a.jpg,231,9,40,21,invoice,416,134
87,106a.jpg,5,/home/scar3crow/Downloads/8-6-new-scan/106a.jpg,323,9,50,22,inv_date,416,134
88,106a.jpg,5,/home/scar3crow/Downloads/8-6-new-scan/106a.jpg,230,63,89,22,po,416,134
89,106a.jpg,5,/home/scar3crow/Downloads/8-6-new-scan/106a.jpg,1,53,154,72,buyer,416,134


In [39]:
anchors = anchors_wrt_target    ## grid-no.-x, grid_no._y, anchor_no., class_no. = x,y,k,c
img_idx, img_resized, y_true, anchor_mask = parse_data(all_image_line[17], class_num, target_size, anchors)
print('===============================')
print(y_true.shape)
print('================================')
print(y_true[2,2])
print('================================')
print(grid_final[2,2])

2 2 2 0
9 2 0 1
12 2 0 2
9 8 0 3
2 9 2 4
(15, 15, 3, 10)
[[0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.15865384 0.1977612  0.30769232 0.38059703 1.         1.
  0.         0.         0.         0.        ]]
[[2. 2.]
 [2. 2.]
 [2. 2.]]


In [40]:
y_pred = np.random.rand(15,450)
y_pred = np.reshape(y_pred, [15,15,3,10])
y_pred[2,2]

array([[0.43487748, 0.09272044, 0.0390249 , 0.26069487, 0.22265465,
        0.18564138, 0.2278643 , 0.59262076, 0.05990115, 0.57574226],
       [0.35897673, 0.37744268, 0.28782252, 0.35605669, 0.02753171,
        0.77120674, 0.14989155, 0.67797714, 0.02268381, 0.61713271],
       [0.03405847, 0.93673087, 0.89324448, 0.61316689, 0.75676532,
        0.28641744, 0.55525129, 0.03295617, 0.14728204, 0.24865604]])

In [175]:
loss = tf.zeros(1, dtype='float32')
    
    
#    m = y_true.shape[0]
#    m = np.expand_dims(m, axis=-1)
#    mf = K.cast(m, dtype='float32')

    
obj_mask_y = y_true[..., 4:5]
#obj_mask = K.cast(obj_mask_y, dtype='float32')
print(obj_mask_y[2,2], obj_mask_y[2,9])
    
#    true_box_wh = y_true[..., 2:4]
#    weight = 2 - true_box_wh[..., 0] * true_box_wh[..., 1]
#    weight = np.expand_dims(weight, axis=-1)
    
## ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++    
pred_box_xy = K.sigmoid(y_pred[..., :2])
pred_box_xy = K.eval(pred_box_xy)
print(pred_box_xy[2,2])
print('=======================================')
                                                            
pred_box_xy = pred_box_xy + grid_final  # this gives x & y in no. of cells. x & y w.r.t. target
print(pred_box_xy[2,2])                                                            # image = (x & y in no. of cells) / no. of cells
print('=======================================')        
#    Lambda_Coord = K.cast(Lambda_Coord, dtype = 'float32')
pred_box_xy_wrt_target_image = (pred_box_xy * 32) / 480.
print(pred_box_xy_wrt_target_image[2,2])                                                            
print('=======================================')  
true_box_xy_wrt_target_image = y_true[..., :2]
print(true_box_xy_wrt_target_image[2,2])
    
#$ pred_box_xy_wrt_ti = K.cast(pred_box_xy_wrt_target_image, dtype = 'float32')
#$ true_box_xy_wrt_ti = K.cast(true_box_xy_wrt_target_image, dtype = 'float32')
    
#$ xy_arr = K.cast((true_box_xy_wrt_ti - pred_box_xy_wrt_ti), dtype='float32')
    
#$ xy_loss = K.cast(Lambda_Coord * tf.reduce_sum(tf.square(xy_arr) * obj_mask), dtype = 'float32')
    
print(obj_mask_y.shape, pred_box_xy_wrt_target_image.shape, true_box_xy_wrt_target_image.shape)    
#$ xy_loss = (Lambda_Coord *np.sum(mean_squared_error(true_box_xy_wrt_target_image, pred_box_xy_wrt_target_image) * obj_mask))
    
#    xy_loss = (Lambda_Coord *np.sum(mean_squared_error(true_box_xy_wrt_ti, pred_box_xy_wrt_ti) * obj_mask*weight)) / m
    
#    xy_loss = (Lambda_Coord*tf.reduce_sum(tf.square(true_box_xy_wrt_ti - pred_box_xy_wrt_ti) *obj_mask))/m
#    wh_loss = tf.reduce_sum(tf.square(true_tw_th - pred_tw_th) * object_mask * box_loss_scale * mix_w) / N
    

[[0.]
 [0.]
 [1.]] [[1.]
 [0.]
 [0.]]
[[0.60703776 0.52316352]
 [0.58879271 0.59325616]
 [0.50851379 0.71843883]]
[[2.60703776 2.52316352]
 [2.58879271 2.59325616]
 [2.50851379 2.71843883]]
[[0.17380252 0.1682109 ]
 [0.17258618 0.17288374]
 [0.16723425 0.18122926]]
[[0.         0.        ]
 [0.         0.        ]
 [0.15865384 0.1977612 ]]
(15, 15, 3, 1) (15, 15, 3, 2) (15, 15, 3, 2)


In [176]:
aaaa = true_box_xy_wrt_target_image -pred_box_xy_wrt_target_image
print(aaaa[2,2])

[[-0.17380252 -0.1682109 ]
 [-0.17258618 -0.17288374]
 [-0.00858041  0.01653194]]


In [177]:
aaa = true_box_xy_wrt_target_image -(pred_box_xy_wrt_target_image * obj_mask_y)
print(aaa[2,2])

[[ 0.          0.        ]
 [ 0.          0.        ]
 [-0.00858041  0.01653194]]


In [178]:
bbbb = np.power(aaaa, 2)
print(bbbb[2,2])


[[3.02073151e-02 2.82949073e-02]
 [2.97859897e-02 2.98887890e-02]
 [7.36234797e-05 2.73304950e-04]]


In [77]:
bbb = np.power(aaa, 2)
print(bbb[2,2])


[[0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00]
 [7.36234797e-05 2.73304950e-04]]


In [73]:
cccc = bbbb * obj_mask_y
print(cccc[2,2])
print('=========================')
print(cccc[2,9])
print('=========================')
print(cccc[2,12])
print('=========================')
print(cccc[8, 9])
print('=========================')
print(cccc[9, 2])
print('=========================')

print(cccc.shape)

[[0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00]
 [7.36234797e-05 2.73304950e-04]]
[[0.18302608 0.2386279 ]
 [0.         0.        ]
 [0.         0.        ]]
[[0.44309909 0.46891106]
 [0.         0.        ]
 [0.         0.        ]]
[[0.00828158 0.00866909]
 [0.         0.        ]
 [0.         0.        ]]
[[0.         0.        ]
 [0.         0.        ]
 [0.20149265 0.23503648]]
(15, 15, 3, 2)


In [78]:

print(bbb[2,2])
print('=========================')
print(bbb[2,9])
print('=========================')
print(bbb[2,12])
print('=========================')
print(bbb[8, 9])
print('=========================')
print(bbb[9, 2])
print('=========================')

print(bbb.shape)

[[0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00]
 [7.36234797e-05 2.73304950e-04]]
[[0.18302608 0.2386279 ]
 [0.         0.        ]
 [0.         0.        ]]
[[0.44309909 0.46891106]
 [0.         0.        ]
 [0.         0.        ]]
[[0.00828158 0.00866909]
 [0.         0.        ]
 [0.         0.        ]]
[[0.         0.        ]
 [0.         0.        ]
 [0.20149265 0.23503648]]
(15, 15, 3, 2)


In [79]:
dddd = np.sum(cccc)
dddd

1.7874908516137276

In [80]:
ddd = np.sum(bbb)
ddd

1.7874908516137276

In [49]:
print(cccc[2,3])
print('=========================')
print(cccc[2,10])
print('=========================')
print(cccc[2,11])
print('=========================')
print(cccc[8, 10])
print('=========================')
print(cccc[9, 3])
print('=========================')

print(cccc.shape)

[[0. 0.]
 [0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]
 [0. 0.]]
[[0. 0.]
 [0. 0.]
 [0. 0.]]
(15, 15, 3, 2)


In [55]:
loss = tf.zeros(1, dtype='float32')
    
    
#    m = y_true.shape[0]
#    m = np.expand_dims(m, axis=-1)
#    mf = K.cast(m, dtype='float32')

    
obj_mask_y = y_true[..., 4:5]
#obj_mask = K.cast(obj_mask_y, dtype='float32')
print(obj_mask_y[2,2], obj_mask_y[2,9])
    
#    true_box_wh = y_true[..., 2:4]
#    weight = 2 - true_box_wh[..., 0] * true_box_wh[..., 1]
#    weight = np.expand_dims(weight, axis=-1)
    
## ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++    


## ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++    
pred_box_wdht = np.exp(y_pred[..., 2:4])
print(pred_box_wdht[2,2])
print('=======================================')
anchors = np.reshape(anchors_wrt_target, [1,1,3,2])/480.
print(anchors)
print('=======================================')
pred_box_wdht = np.exp(y_pred[..., 2:4]) * (anchors_wrt_target/480.)
print(pred_box_wdht[2,2])
print('=======================================')
#$    pred_box_wh = K.cast(pred_box_wdht, dtype = 'float32')
true_box_wdht = y_true[..., 2:4]
#$    true_box_wh = K.cast(true_box_wdht, dtype = 'float32')
print(true_box_wdht[2,2])
print('=======================================')
    
#    wh_arr = K.cast((K.sqrt(true_box_wh) - K.sqrt(pred_box_wh)), dtype='float32')
    
#    wh_loss = K.cast(Lambda_Coord * tf.reduce_sum(tf.square(wh_arr) * obj_mask), dtype = 'float32')
    
## ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++ 



[[0.]
 [0.]
 [1.]] [[1.]
 [0.]
 [0.]]
[[1.03979638 1.29783159]
 [1.3335206  1.42768849]
 [2.44304321 1.84626909]]
[[[[0.15625    0.11904762]
   [0.5625     0.26746644]
   [0.37980769 0.36309524]]]]
[[0.16246818 0.15450376]
 [0.75010534 0.38185875]
 [0.92788661 0.67037151]]
[[0.         0.        ]
 [0.         0.        ]
 [0.30769232 0.38059703]]


In [61]:
ffff = true_box_wdht - pred_box_wdht
print(ffff[2,2])

[[-0.16246818 -0.15450376]
 [-0.75010534 -0.38185875]
 [-0.62019429 -0.28977449]]


In [62]:
gggg = np.power(ffff, 2)
print(gggg[2,2])


[[0.02639591 0.02387141]
 [0.56265802 0.14581611]
 [0.38464095 0.08396925]]


In [65]:
hhhh = gggg * obj_mask_y
print(hhhh[2,2])
print('=========================')
print(hhhh[2,9])
print('=========================')
print(hhhh[2,12])
print('=========================')
print(hhhh[8, 9])
print('=========================')
print(hhhh[9, 2])
print('=========================')

print(hhhh.shape)

[[0.         0.        ]
 [0.         0.        ]
 [0.38464095 0.08396925]]
[[0.09631664 0.0145423 ]
 [0.         0.        ]
 [0.         0.        ]]
[[0.00146837 0.0007298 ]
 [0.         0.        ]
 [0.         0.        ]]
[[0.00877394 0.00042433]
 [0.         0.        ]
 [0.         0.        ]]
[[0.         0.        ]
 [0.         0.        ]
 [0.26326866 0.11321882]]
(15, 15, 3, 2)


In [66]:
jjjj = np.sum(hhhh)
jjjj

0.967353063197183

In [87]:
# https://github.com/ethanyanjiali/deep-vision/blob/master/YOLO/tensorflow/utils.py

def xywh_go_x1y1x2y2(box):
    xy = box[..., 0:2]
    wh = box[..., 2:4]

    x1y1 = xy - wh / 2
    x2y2 = xy + wh / 2

    y_box = np.concatenate([x1y1, x2y2], axis=-1)
    return y_box

In [91]:

pred_obj = y_pred[..., 4:5]
print(pred_obj[2,2])
print('============================================================')

pred_obj_mask = my_sigmoid(y_pred[..., 4:5])  # shape = 28, 15, 15, 3, 1

print(pred_obj_mask[2,2])
print('============================================================')

print(true_box_xy_wrt_target_image[2, 2])
print(true_box_wdht[2,2])
print('============================================================')

true_box_wrt_ti = np.concatenate([true_box_xy_wrt_target_image, true_box_wdht], axis = -1)  ## in x,y,w,h format
print(true_box_wrt_ti[2,2])
print('============================================================')

pred_box_wrt_ti = np.concatenate([pred_box_xy_wrt_target_image, pred_box_wdht], axis = -1)  ## in x,y,w,h format
print(pred_box_wrt_ti[2,2])
print('============================================================')
    
true_box_wrt_ti = xywh_go_x1y1x2y2(true_box_wrt_ti)  ## converted to x1,y1,x2,y2 format
pred_box_wrt_ti = xywh_go_x1y1x2y2(pred_box_wrt_ti)  ## converted to x1,y1,x2,y2 format

print(true_box_wrt_ti[2,2])
print('============================================================')
print(pred_box_wrt_ti[2,2])
print('============================================================')

print(true_box_wrt_ti.shape)




    
#    ignore_mask = calc_ignore_mask(ignore_thresh, true_box_wrt_ti, pred_box_wrt_ti)
    
#    ignore_mask = K.cast(ignore_mask, dtype = 'float32')                               

#    obj_loss = K.cast(K.sum(K.binary_crossentropy(obj_mask, pred_obj_mask) * obj_mask), dtype = 'float32')
    
#    no_obj_mask = 1. - obj_mask_y
#    no_obj_mask = K.cast(no_obj_mask, dtype  = 'float32')
    
#    noobj_loss = K.cast(Lambda_no_obj * K.sum(binary_cross_entropy(obj_mask, pred_obj_mask) * no_obj_mask * ignore_mask), dtype='float32')
    

[[0.22265465]
 [0.02753171]
 [0.75676532]]
[[0.55543484]
 [0.50688249]
 [0.68065104]]
[[0.         0.        ]
 [0.         0.        ]
 [0.15865384 0.1977612 ]]
[[0.         0.        ]
 [0.         0.        ]
 [0.30769232 0.38059703]]
[[0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.15865384 0.1977612  0.30769232 0.38059703]]
[[0.17380252 0.1682109  0.16246818 0.15450376]
 [0.17258618 0.17288374 0.75010534 0.38185875]
 [0.16723425 0.18122926 0.92788661 0.67037151]]
[[0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.00480768 0.00746268 0.3125     0.3880597 ]]
[[ 0.09256843  0.09095902  0.25503661  0.24546278]
 [-0.20246649 -0.01804563  0.54763885  0.36381312]
 [-0.29670905 -0.1539565   0.63117756  0.51641501]]
(15, 15, 3, 4)


In [ ]:
def broadcast_iou(box_a, box_b):
    """
    calculate iou between box_a and multiple box_b in a broadcast way
    inputs: box_a: a tensor full of boxes, eg. (B, N, 4), box is in x1y1x2y2
            box_b: another tensor full of boxes, eg. (B, M, 4)
    """

    # (B, N, 1, 4)
    box_a = tf.expand_dims(box_a, -2)
    # (B, 1, M, 4)
    box_b = tf.expand_dims(box_b, -3)
    # (B, N, M, 4)
    new_shape = tf.broadcast_dynamic_shape(tf.shape(box_a), tf.shape(box_b))

    # (B, N, M, 4)
    # (B, N, M, 4)
    box_a = tf.broadcast_to(box_a, new_shape)
    box_b = tf.broadcast_to(box_b, new_shape)

    # (B, N, M, 1)
    al, at, ar, ab = tf.split(box_a, 4, -1)
    bl, bt, br, bb = tf.split(box_b, 4, -1)

    # (B, N, M, 1)
    left = tf.math.maximum(al, bl)
    right = tf.math.minimum(ar, br)
    top = tf.math.maximum(at, bt)
    bot = tf.math.minimum(ab, bb)

    # (B, N, M, 1)
    iw = tf.clip_by_value(right - left, 0, 1)
    ih = tf.clip_by_value(bot - top, 0, 1)
    i = iw * ih

    # (B, N, M, 1)
    area_a = (ar - al) * (ab - at)
    area_b = (br - bl) * (bb - bt)
    union = area_a + area_b - i

    # (B, N, M)
    iou = tf.squeeze(i / (union + 1e-7), axis=-1)

    return iou

def calc_ignore_mask(ignore_thresh, true_box, pred_box):
    
        # YOLOv3:
        # "If the bounding box prior is not the best but does overlap a ground
        # truth object by more than some threshold we ignore the prediction,
        # following [17]. We use the threshold of .5."
        # calculate the iou for each pair of pred bbox and true bbox, then find the best among them

        # (None, 13, 13, 3, 4)
        
        true_box_reorganised = xywh_to_x1y1x2y2(true_box)  # reorganised to x1, y1, x2, y2
        pred_box_reorganised = xywh_to_x1y1x2y2(pred_box)
        
        true_box_shape = tf.shape(true_box_reorganised)  
        # (None, 13, 13, 3, 4)
        pred_box_shape = tf.shape(pred_box_reorganised)  
        # (None, 507, 4)
        true_box_reorganised = tf.reshape(true_box_reorganised, [true_box_shape[0], -1, 4])
        # sort true_box to have non-zero boxes rank first
        true_box_reorganised = tf.sort(true_box_reorganised, axis=1, direction="DESCENDING")
        # (None, 100, 4)
        # only use maximum 100 boxes per groundtruth to calcualte IOU, otherwise
        # GPU emory comsumption would explode for a matrix like (16, 52*52*3, 52*52*3, 4)
        true_box_reorganised = true_box_reorganised[:, 0:100, :]
        # (None, 507, 4)
        pred_box_reorganised = tf.reshape(pred_box_reorganised, [pred_box_shape[0], -1, 4])

        # https://github.com/dmlc/gluon-cv/blob/06bb7ec2044cdf3f433721be9362ab84b02c5a90/gluoncv/model_zoo/yolo/yolo_target.py#L198
        # (None, 507, 507)
        iou = broadcast_iou(pred_box_reorganised, true_box_reorganised)
        # (None, 507)
        best_iou = tf.reduce_max(iou, axis=-1)
        # (None, 13, 13, 3)
        best_iou = tf.reshape(best_iou, [pred_box_shape[0], pred_box_shape[1], pred_box_shape[2], pred_box_shape[3]])
        # ignore_mask = 1 => don't ignore
        # ignore_mask = 0 => should ignore
        ignore_mask = tf.cast(best_iou < ignore_thresh, tf.float32)
        # (None, 13, 13, 3, 1)
        ignore_mask = tf.expand_dims(ignore_mask, axis=-1)
        
        return ignore_mask

In [166]:
#$ ignore_mask = calc_ignore_mask(ignore_thresh, true_box_wrt_ti, pred_box_wrt_ti)
print(true_box_wrt_ti.shape)  # shape = 15, 15, 3, 4
print(true_box_wrt_ti[2,2])  # (2,2-anchor-2), (2,9-anchor-0), (2,12-anchor-0), (8,9-anchor-0), (9,2-anchor-2)
print('===========================================')
true_box_reorganised = np.reshape(true_box_wrt_ti, [true_box_wrt_ti.shape[0], -1, 4]) # change shape to 15, 45, 4
print(true_box_reorganised.shape)
print(true_box_reorganised[2,2*3+2])  # '+2' since anchor[2]
print(true_box_reorganised[8, 9*3+0]) # '+0' since anchor[0]
print('===========================================')
inter_true = tf.sort(true_box_reorganised, axis=1, direction="DESCENDING") # sorting in descending order
true_box_reorganised = K.eval(inter_true)
print(true_box_reorganised.shape)
print(true_box_reorganised[2,0])
print(true_box_reorganised[8,0])
print('===========================================')
print(pred_box_wrt_ti.shape)  # shape = 15, 15, 3, 4
print(pred_box_wrt_ti[2,2])  # (2,2-anchor-2), (2,9-anchor-0), (2,12-anchor-0), (8,9-anchor-0), (9,2-anchor-2)
print('===========================================')

pred_box_reorganised = np.reshape(pred_box_wrt_ti, [pred_box_wrt_ti.shape[0], -1, 4])
print(pred_box_reorganised.shape)
print('++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')

#$$$$$  iou = broadcast_iou(pred_box_reorganised, true_box_reorganised)

# (B, N, 1, 4)

box_a = pred_box_reorganised
box_b = true_box_reorganised
print(box_b.shape)
print('===========================================')

box_a = np.expand_dims(box_a, -2)
    # (B, 1, M, 4)
box_b = np.expand_dims(box_b, -3)
print('pred_box shape = ', box_a.shape)
print('===========================================')
print('true_box shape = ', box_b.shape)
print


    # (B, N, M, 4)
new_shape_inter = tf.broadcast_dynamic_shape(tf.shape(box_a), tf.shape(box_b))
new_shape = K.eval(new_shape_inter)
print('new shape = ', new_shape)

    # (B, N, M, 4)
    # (B, N, M, 4)
box_a = tf.broadcast_to(box_a, new_shape)
box_b = tf.broadcast_to(box_b, new_shape)

print('true_box shape = ', K.eval(box_b).shape)
print( K.eval(box_b)[2,0])
print('===========================================')

box_a = K.cast(box_a, dtype = 'float32')
box_b = K.cast(box_b, dtype = 'float32')



    # (B, N, M, 1)
al, at, ar, ab = tf.split(box_a, 4, -1)
bl, bt, br, bb = tf.split(box_b, 4, -1)
#print(K.eval(bl).shape)
#print(K.eval(bl)[2,0])

    # (B, N, M, 1)
left = tf.math.maximum(al, bl)
right = tf.math.minimum(ar, br)
top = tf.math.maximum(at, bt)
bot = tf.math.minimum(ab, bb)

    # (B, N, M, 1)
iw = tf.clip_by_value(right - left, 0, 1)
ih = tf.clip_by_value(bot - top, 0, 1)
i = iw * ih

    # (B, N, M, 1)
area_a = (ar - al) * (ab - at)
area_b = (br - bl) * (bb - bt)
union = area_a + area_b - i

    # (B, N, M)
iou = tf.squeeze(i / (union + 1e-7), axis=-1)
print(K.eval(iou).shape)
print(K.eval(iou)[2,0])

best_iou = tf.reduce_max(iou, axis=-1)
print('best = ', K.eval(best_iou).shape)
print('best = ', K.eval(best_iou))

        # (None, 13, 13, 3)
#        best_iou = tf.reshape(best_iou, [pred_box_shape[0], pred_box_shape[1], pred_box_shape[2], pred_box_shape[3]])
        # ignore_mask = 1 => don't ignore
        # ignore_mask = 0 => should ignore
#        ignore_mask = tf.cast(best_iou < ignore_thresh, tf.float32)
        # (None, 13, 13, 3, 1)
#        ignore_mask = tf.expand_dims(ignore_mask, axis=-1)






#    ignore_mask = K.cast(ignore_mask, dtype = 'float32')                               

#    obj_loss = K.cast(K.sum(K.binary_crossentropy(obj_mask, pred_obj_mask) * obj_mask), dtype = 'float32')
    
#    no_obj_mask = 1. - obj_mask_y
#    no_obj_mask = K.cast(no_obj_mask, dtype  = 'float32')
    
#    noobj_loss = K.cast(Lambda_no_obj * K.sum(binary_cross_entropy(obj_mask, pred_obj_mask) * no_obj_mask * ignore_mask), dtype='float32')
    

(15, 15, 3, 4)
[[0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.00480768 0.00746268 0.3125     0.3880597 ]]
(15, 45, 4)
[0.00480768 0.00746268 0.3125     0.3880597 ]
[0.55288464 0.47014928 0.7668269  0.63432837]
(15, 45, 4)
[0.7764423  0.06716419 0.8966346  0.3880597 ]
[0.55288464 0.47014928 0.7668269  0.63432837]
(15, 15, 3, 4)
[[ 0.09256843  0.09095902  0.25503661  0.24546278]
 [-0.20246649 -0.01804563  0.54763885  0.36381312]
 [-0.29670905 -0.1539565   0.63117756  0.51641501]]
(15, 45, 4)
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
(15, 45, 4)
pred_box shape =  (15, 45, 1, 4)
true_box shape =  (15, 1, 45, 4)
new shape =  [15 45 45  4]
true_box shape =  (15, 45, 45, 4)
[[0.7764423  0.06716419 0.8966346  0.3880597 ]
 [0.55528843 0.06716419 0.6514423  0.2313433 ]
 [0.00480768 0.00746268 0.3125     0.22388062]
 [0.         0.         0.         0.        ]
 [0.         0.         0.        

In [171]:
rrrr = K.eval(best_iou)
rrrr[9]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.0279248 , 0.04215166, 0.        ,
       0.0228469 , 0.01498918, 0.        , 0.05393586, 0.        ,
       0.        , 0.10110861, 0.08518045, 0.        , 0.12846251,
       0.        , 0.        , 0.11978302, 0.        , 0.        ,
       0.2082091 , 0.08403183, 0.        , 0.09204015, 0.        ,
       0.        , 0.21353522, 0.08229756, 0.        , 0.15810756,
       0.09590296, 0.        , 0.16865684, 0.04816975, 0.        ,
       0.03479904, 0.02273013, 0.        , 0.08595777, 0.        ],
      dtype=float32)

In [100]:
true_box_reorganised[2]  ## result before sorting

array([[0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.00480768, 0.00746268, 0.3125    , 0.3880597 ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        ,

In [135]:
qqqq[2].shape

(4, 45, 4)

In [134]:
true_box_reorganised[2].shape

(45, 4)

In [137]:
a = [1, 10, 26.9, 2.8, 166.32, 62.3]
b = tf.sort(a,axis=-1,direction='ASCENDING',name=None)
c = tf.keras.backend.eval(b)
c

array([  1.  ,   2.8 ,  10.  ,  26.9 ,  62.3 , 166.32], dtype=float32)

In [144]:
a = true_box_reorganised
b = tf.sort(a,axis=1,direction='DESCENDING',name=None)
c = K.eval(b)
print(a.shape)
print(c.shape)

(15, 45, 4)
(15, 45, 4)


In [147]:
c[8]

array([[0.55288464, 0.47014928, 0.7668269 , 0.63432837],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        ,

In [143]:
print(type(a), type(c))

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
